# İki Konum Arasındaki Mesafe, Sürüş Mesafesi ve Sürenin Gezgin Satıcı Problemi Genetik Algoritma Yöntemi İle Hesaplanması 

## Paketlerin ve Kütüphanelerin İndirilmesi

In [1]:
#sklearn.externals.six paketinin hatasının çözümü maksadıyla ayrıca modül indirimi yapılmıştır.
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose

In [2]:
import pandas as pd
from geopy import distance
import requests # to call the openmap/google apis
import json
import datetime
import math
import itertools

# Eğer sklearn.external six paketi ile problemle karşılaşır iseniz yardımcı link: https://stackoverflow.com/questions/61867945/python-import-error-cannot-import-name-six-from-sklearn-externals
# Normalde original paket indirimi şöyle yapılır:
#from sklearn.external import six
# to:
# import six
import mlrose #travelling salesman problem çözümü için

# that was fixed in mlrose_hiive, though the outputs are different
# import mlrose_hiive
import numpy as np


In [3]:
# Veri yüklenmesi yapılmıştır.(Dünya başkentlerine ait ülke ismi,başkent,enlem,boylam,kodları,kıta ismi verileri olan GPS)
df = pd.read_csv("concap.csv")

# sütun adlarının daha kısa olması ve PEP-8 ile uyumlu olması için
df.rename(columns={"CountryName": "Country", "CapitalName": "capital", "CapitalLatitude": "lat", "CapitalLongitude": "lon", "CountryCode": "code", "ContinentName": "continent"}, inplace=True)
df.head(10)

,Country,capital,lat,lon,code,continent
0,Somaliland,Hargeisa,9.550000,44.050000,NaN,Africa
1,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,GS,Antarctica
2,French Southern and Antarctic Lands,Port-aux-Français,-49.350000,70.216667,TF,Antarctica
3,Palestine,Jerusalem,31.766667,35.233333,PS,Asia
4,Aland Islands,Mariehamn,60.116667,19.900000,AX,Europe
5,Nauru,Yaren,-0.547700,166.920867,NR,Australia
6,Saint Martin,Marigot,18.073100,-63.082200,MF,North America
7,Tokelau,Atafu,-9.166667,-171.833333,TK,Australia
8,Western Sahara,El-Aaiún,27.153611,-13.203333,EH,Africa
9,Afghanistan,Kabul,34.516667,69.183333,AF,Asia


In [4]:

# Başlangıcı üç dünya başkentini filitreliyerek başlayalım.
ropa = df[df["capital"].isin(["Ankara","London","Copenhagen"])].reset_index()
cities = ropa.copy()
cities

,index,Country,capital,lat,lon,code,continent
0,66,Denmark,Copenhagen,55.666667,12.583333,DK,Europe
1,219,Turkey,Ankara,39.933333,32.866667,TR,Europe
2,226,United Kingdom,London,51.500000,-0.083333,GB,Europe


## Mesafenin Hesaplanması

İlk belirgin yöntem, Dünya başkentleri arasındaki, en kısa mesafeyi bulmaktdır. Çeşitli yaklaşımlar kullanılabilir

Bu analizi tekrardan detaylı bir şekilde derinlemesine sıfırdan oluşturmamıza ve formüle etmemize gerek bırakmayan geopy uzaklık hesaplama modülü tüm bu mesafe hesaplamasını zaten uygulamaktadır.Kilometre (km), mil (mil), deniz mili (nm) veya fit (ft) cinsinden değerleri hesaplar. Tüm bu yöntemler, geopy'den içe aktardığımız uzaklık sınıfının bir parçasıdır.

distance((latitude_point_1, longitude_point_1), (lat_2, lon_2)) - using geodesic on WGS-84 ellipsoid
geodesic((latitude_point_1, longitude_point_1), (lat_2, lon_2))
great_circle((latitude_point_1, longitude_point_1), (lat_2, lon_2))

In [126]:
d = distance.distance((cities.loc[0, "lat"], cities.loc[0, "lon"]), (cities.loc[1, "lat"], cities.loc[1, "lon"]))
d, d.km, d.miles

(Distance(2299.5504218172364), 2299.5504218172364, 1428.8743872144403)

In [127]:
getattr(d, "km")

2299.5504218172364

In [128]:
results = []
for f in [distance.distance, distance.great_circle, distance.geodesic]:
    for mes in ["kilometers","km","miles","mi","nautical","nm","feet","ft"]:
        d = f((cities.loc[0, "lat"], cities.loc[0, "lon"]), (cities.loc[1, "lat"], cities.loc[1, "lon"]))
        results.append({"method": f.__name__, "measurement": mes, "value": getattr(d, mes)})

# show as dataframe
results_df = pd.DataFrame(results)
results_df.pivot_table(index="method", columns="measurement", values="value")

measurement,feet,ft,kilometers,km,mi,miles,nautical,nm
method,,,,,,,,
geodesic,7.544457e+06,7.544457e+06,2299.550422,2299.550422,1428.874387,1428.874387,1241.657895,1241.657895
great_circle,7.535520e+06,7.535520e+06,2296.826600,2296.826600,1427.181883,1427.181883,1240.187149,1240.187149


Distance.distance doğal olarak Distance.geodesic'i çağırır, bu yüzden bu iki değer sonuçlarda bir satıra sığar.

In [129]:

# the distnace for various ellipsiods
for ellipsoid in distance.ELLIPSOIDS:
    for mes in ["kilometers","km","miles","mi","nautical","nm","feet","ft"]:
        d = distance.geodesic((cities.loc[0, "lat"], cities.loc[0, "lon"]), (cities.loc[1, "lat"], cities.loc[1, "lon"]), ellipsoid=ellipsoid)
        results.append({"method": f"geodesic: {ellipsoid}", "measurement": mes, "value": getattr(d, mes)})

# show as dataframe
results_df = pd.DataFrame(results)
results_df.pivot_table(index="method", columns="measurement", values="value")

measurement,feet,ft,kilometers,km,mi,miles,nautical,nm
method,,,,,,,,
geodesic,7.544457e+06,7.544457e+06,2299.550422,2299.550422,1428.874387,1428.874387,1241.657895,1241.657895
geodesic: Airy (1830),7.543777e+06,7.543777e+06,2299.343094,2299.343094,1428.745560,1428.745560,1241.545947,1241.545947
geodesic: Clarke (1880),7.544597e+06,7.544597e+06,2299.593244,2299.593244,1428.900996,1428.900996,1241.681017,1241.681017
geodesic: GRS-67,7.544484e+06,7.544484e+06,2299.558718,2299.558718,1428.879542,1428.879542,1241.662375,1241.662375
geodesic: GRS-80,7.544457e+06,7.544457e+06,2299.550422,2299.550422,1428.874387,1428.874387,1241.657895,1241.657895
geodesic: Intl 1924,7.544756e+06,7.544756e+06,2299.641536,2299.641536,1428.931003,1428.931003,1241.707093,1241.707093
geodesic: WGS-84,7.544457e+06,7.544457e+06,2299.550422,2299.550422,1428.874387,1428.874387,1241.657895,1241.657895
great_circle,7.535520e+06,7.535520e+06,2296.826600,2296.826600,1427.181883,1427.181883,1240.187149,1240.187149


## Sürüş Mesafesi

Şehirler yüzeyde oldukça yakın olabilir, ancak deniz veya dağ gibi doğal engeller sürüş mesafesinin çok daha uzun olmasına neden olabilir.

In [130]:

hest = df[df["capital"].isin(["Helsinki","Stockholm"])].reset_index()
cities = hest.copy()
d = distance.distance((cities.loc[0, "lat"], cities.loc[0, "lon"]), (cities.loc[1, "lat"], cities.loc[1, "lon"]))
d.km

397.76330968599365

Finlandiya'nın başkenti Helsinky ile İsveç'teki Stockholm arasındaki mesafe 400 km'den az olsa da, gitmeye(kara yolu ile) karar verirseniz 1750km'den ve 20 saatten fazla. Feribotla gitseniz bile neredeyse 500 km yol gideceksiniz. Paris, Roma'dan sadece 1107km uzaklıkta, ancak bu şehirleri birbirine bağlayan yollar en az 1420 km uzunluğunda.

Bu nedenle, birçok uygulama için  standart matematiksel işlemlerle hesaplanması mümkün olmayan geri dönüş optimizasyonunuda hesaplayan gerçek seyahat mesafesini bilmek istersiniz.

Bazı harita hizmeti API'lerini çağırarak projemizi ilerlettik - ör. google rotaları veya osrm rota hizmeti gibi  (http://project-osrm.org/docs/v5.5.1/api/#route-service)

Belgelerden elde edilen döküman sonuçlarından, sürüş mesafesini elde etmek için bu API endointini çağırmamız gerektiğinine ulaşılmıştır.

/route/v1/{profile}/{coordinates}?alternatives={true|false}&steps={true|false}&geometries={polyline|polyline6|geojson}&overview={full|simplified|false}&annotations={true|false} sahip parametreler çağrılmıştır:
profile - car, bike, foot
coordinates - lat,lon of the first point; lon, lat of the second point, e.g. 2.333333,48.866667;12.483333,41.900000
alternative - yalnızca ilk seçeneğin mi yoksa daha fazla alternatifin mi döndürüleceğini,
steps - rota adımlarının döndürülüp döndürülmeyeceği - ör. kavşakta sola dön gibi,
geometries -yol nasıl döndürülür, ya polyline (varsayılan), polyline6, geojson`,
overview - rota nasıl döndürülür - simplified (default), full, false
annotations -rotadaki her nokta için ek meta veriler sağlanmışsa

Süreç dahilinde, her şeyi yanlış veya varsayılan olarak ayarlayarak en basit isteği çalıştıracağız. Biz sadece çağırarak bütünleşik bir yapı ortaya koymuş oluyoruz:
http://router.project-osrm.org/route/v1/driving/cities.loc[0, "lon"],cities.loc[0, "lat"];cities.loc[1, "lon"],cities.loc[1, "lat"]?overview=false

#### API'nin yanıtını almak için Python request yöntemi kulanılmıştır.

In [131]:
cities = ropa.copy()
r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{cities.loc[0, "lon"]},{cities.loc[0, "lat"]};{cities.loc[1, "lon"]},{cities.loc[1, "lat"]}?overview=false""")

#### API tarafından içerik parametresinde sağlananları döndürür.

In [132]:
r.content

b'{"code":"Ok","waypoints":[{"hint":"tzqEi8w6hIsPAAAAJQAAAAAAAAAAAAAAGMXVQMVVc0EAAAAAAAAAAA8AAAAlAAAAAAAAAAAAAAD97QAA0gHAANdnUQOlAcAA62dRAwAAXwLgRcQ5","distance":3.60203,"location":[12.583378,55.666647],"name":""},{"hint":"SrjLgP___38hAAAAIwAAAPgAAAAmAAAA-HpuQjlEV0DqJeBDJHuKQiEAAAAjAAAA-AAAACYAAAD97QAAfoH1AfhVYQJrgfUBlVVhAg4ALwbgRcQ5","distance":11.111453,"location":[32.866686,39.933432],"name":"Talat Pa\xc5\x9fa Bulvar\xc4\xb1"}],"routes":[{"legs":[{"steps":[],"weight":125890.8,"distance":3394543.3,"summary":"","duration":125890.8}],"weight_name":"routability","weight":125890.8,"distance":3394543.3,"duration":125890.8}]}'

#### Python'un json kitaplığına geçirildiğinde güzel görünen bir json nesnesi döndürdüğü görülmektedir.

In [133]:
json.loads(r.content)

{'code': 'Ok',
 'waypoints': [{'hint': 'tzqEi8w6hIsPAAAAJQAAAAAAAAAAAAAAGMXVQMVVc0EAAAAAAAAAAA8AAAAlAAAAAAAAAAAAAAD97QAA0gHAANdnUQOlAcAA62dRAwAAXwLgRcQ5',
   'distance': 3.60203,
   'location': [12.583378, 55.666647],
   'name': ''},
  {'hint': 'SrjLgP___38hAAAAIwAAAPgAAAAmAAAA-HpuQjlEV0DqJeBDJHuKQiEAAAAjAAAA-AAAACYAAAD97QAAfoH1AfhVYQJrgfUBlVVhAg4ALwbgRcQ5',
   'distance': 11.111453,
   'location': [32.866686, 39.933432],
   'name': 'Talat Paşa Bulvarı'}],
 'routes': [{'legs': [{'steps': [],
     'weight': 125890.8,
     'distance': 3394543.3,
     'summary': '',
     'duration': 125890.8}],
   'weight_name': 'routability',
   'weight': 125890.8,
   'distance': 3394543.3,
   'duration': 125890.8}]}

Esas olarak sürüş mesafesi ve/veya sürüş süresi ile ilgileniyoruz. Bu parametreler, bir rota listesi içeren rota alt öğesine dahil edilir. Herhangi bir alternatif aramadığımız için bu listede sadece bir madde var.

In [134]:
route_1 = json.loads(r.content)["routes"][0]
route_1["distance"], route_1["duration"]

(3394543.3, 125890.8)

Ortaya çıkan sayılar garip görünüyorsa, mesafenin metre cinsinden ve sürenin saniye cinsinden olduğuna dikkat edilmesi gerekmektedir.Daha iyi okunabilir bir formata çevirebilmek için süreç işletilmiştir. Bu, alınan süreyi bir parametre olarak datetime'ın timedelta işlevine geçirerek ve dize temsilini döndürerek elde edilmiştir.(1 günden fazla olması durumunda (bu amaçla Paris-Roma mesafesini 100000 saniye eklenmiştir)).

In [135]:
x = datetime.timedelta(seconds=route_1["duration"])

In [136]:
str(x)

'1 day, 10:58:10.800000'

Süre sütununun türünü timedelta64[s] olarak belirtmemizin sebebi, panda kütüphanesinin işlevsel olarak kullanılabilmesi amacıyladır.

In [137]:
dftime = pd.DataFrame({"duration":route_1["duration"]+100000}, index=[0])
dftime["duration"] = dftime["duration"].astype("timedelta64[s]")
dftime

,duration
0,2 days 14:44:50


In [138]:
origin_coor = ",".join([str(cities.loc[0,"lat"]), str(cities.loc[0,"lon"])])
destination_coor = ",".join([str(cities.loc[1,"lat"]), str(cities.loc[1,"lon"])])
API_KEY = "...your_google_api_key..."
url = f"https://maps.googleapis.com/maps/api/directions/json?origin={origin_coor}&destination={destination_coor}&mode=driving&key={API_KEY}"

# alternatively you can specify the start point (origin) and the destination using the places' names
url_alt = f"https://maps.googleapis.com/maps/api/directions/json?origin=Paris&destination=Rome&mode=driving&key={API_KEY}"
print(url)

https://maps.googleapis.com/maps/api/directions/json?origin=55.66666666666666,12.583333&destination=39.93333333333333,32.866667&mode=driving&key=...your_google_api_key...


In [139]:
r = requests.get(url_alt)

Bu durumda, sonuçları yanıtın uygun değerlerinde bulabileceğiz.

## Birden fazla şehir ve optimum rota arasındaki mesafeler

In [140]:
ce_countries = ["AT","CZ","DE","HU","LI","PL","SK","SI","CH"]

In [141]:
ce_cities = df[df["code"].isin(ce_countries)].reset_index(drop=True)
ce_cities

,Country,capital,lat,lon,code,continent
0,Austria,Vienna,48.200000,16.366667,AT,Europe
1,Czech Republic,Prague,50.083333,14.466667,CZ,Europe
2,Germany,Berlin,52.516667,13.400000,DE,Europe
3,Hungary,Budapest,47.500000,19.083333,HU,Europe
4,Liechtenstein,Vaduz,47.133333,9.516667,LI,Europe
5,Poland,Warsaw,52.250000,21.000000,PL,Europe
6,Slovakia,Bratislava,48.150000,17.116667,SK,Europe
7,Slovenia,Ljubljana,46.050000,14.516667,SI,Europe
8,Switzerland,Bern,46.916667,7.466667,CH,Europe


OSRM uzaklık değerini  fonksiyona çevrilmiştir.

In [142]:
def get_distance(point1: dict, point2: dict) -> tuple:
    """Gets distance between two points en route using http://project-osrm.org/docs/v5.10.0/api/#nearest-service"""
    
    url = f"""http://router.project-osrm.org/route/v1/driving/{point1["lon"]},{point1["lat"]};{point2["lon"]},{point2["lat"]}?overview=false&alternatives=false"""
    r = requests.get(url)
    
    # get the distance from the returned values
    route = json.loads(r.content)["routes"][0]
    return (route["distance"], route["duration"])

In [143]:
# İki şehir için deneme yapılmıştır.
# bağlantıyı teyit edelim şu adresten: https://www.openstreetmap.org/directions?engine=fossgis_osrm_car&route=48.208%2C16.372%3B50.087%2C14.421
get_distance({"lat": 48.200000,"lon": 16.366667}, {"lat": 50.083333,"lon": 14.466667})

(333871.2, 13485.7)

Artık şehirlerin tüm kombinasyonları için mesafe hesaplamasını yapabiliriz.9 a 2 kombinasyon yapılmıştır.

In [144]:
combination = itertools.combinations(list(ce_cities["capital"]),2)
len([c for c in itertools.combinations(list(ce_cities["capital"]),2)])

36

In [145]:
list(ce_cities["capital"])

['Vienna',
 'Prague',
 'Berlin',
 'Budapest',
 'Vaduz',
 'Warsaw',
 'Bratislava',
 'Ljubljana',
 'Bern']

In [146]:
math.comb(9,2)

36

Tüm kombinasyonlarımız için OSRM API'den mesafe ve süreyi alalım.

In [147]:
dist_array = []
for i , r in ce_cities.iterrows():
    point1 = {"lat": r["lat"], "lon": r["lon"]}
    for j, o in ce_cities[ce_cities.index != i].iterrows():
        point2 = {"lat": o["lat"], "lon": o["lon"]}
        dist, duration = get_distance(point1, point2)
        #dist = geodesic((i_lat, i_lon), (o["CapitalLatitude"], o["CapitalLongitude"])).km
        dist_array.append((i, j, duration, dist))

In [148]:
distances_df = pd.DataFrame(dist_array,columns=["origin","destination","duration(s)","distnace(m)"])
distances_df

,origin,destination,duration(s),distnace(m)
0,0,1,13485.7,333871.2
1,0,2,26971.0,681982.9
2,0,3,9627.4,246207.5
3,0,4,24179.8,647086.7
4,0,5,29590.6,672145.8
...,...,...,...,...
67,8,3,42033.8,1102404.1
68,8,4,10012.4,235665.4
69,8,5,52620.7,1498735.9
70,8,6,35890.9,936758.2


In [149]:
distances_df = distances_df.merge(ce_cities[["capital"]], left_on = "origin", right_index=True).rename(columns={"capital":"origin_name"})
distances_df = distances_df.merge(ce_cities[["capital"]], left_on = "destination", right_index=True).rename(columns={"capital":"destination_name"})
distances_df

,origin,destination,duration(s),distnace(m),origin_name,destination_name
0,0,1,13485.7,333871.2,Vienna,Prague
17,2,1,13652.6,348020.9,Berlin,Prague
25,3,1,19593.8,527320.4,Budapest,Prague
33,4,1,22575.3,633999.7,Vaduz,Prague
41,5,1,26700.9,673814.6,Warsaw,Prague
...,...,...,...,...,...,...
32,4,0,24206.8,646715.5,Vaduz,Vienna
40,5,0,29661.1,672305.4,Warsaw,Vienna
48,6,0,3488.7,79230.9,Bratislava,Vienna
56,7,0,14281.6,374125.5,Ljubljana,Vienna


### Gezgin Satıcı Problemi Genetik Algoritma Uygulama Aşaması

Bir parametreye dayalı bir nokta liste arasındaki en uygun rotayı bulmak için Gezgin Satıcı Problemini kullanıyoruz.

Gezgin Satıcı en az çabayla tüm noktaları ziyaret etmek ister.

In [150]:

# 9 şehre ziyaret planlıyoruz.
length = ce_cities.shape[0]

Mesafeler listesini kullanmayı planlıyoruz (bizim durumumuzdaki süreler), bu yüzden mesafeler = dist_list param ile başlatıyoruz. Örneğin. 0 ile 1. yer arasındaki mesafe 13949.0 saniye --> (0,1,13949,0)

In [152]:
# dataframe'in ilk üç sütununu tuples listesine çeviriyoruz
dist_list = list(distances_df[["origin","destination","duration(s)"]].sort_values(by=["origin","destination"]).to_records(index=False))
dist_list[:5] + ["..."] + dist_list[-5:]

[(0, 1, 13485.7),
 (0, 2, 26971.),
 (0, 3, 9627.4),
 (0, 4, 24179.8),
 (0, 5, 29590.6),
 '...',
 (8, 3, 42033.8),
 (8, 4, 10012.4),
 (8, 5, 52620.7),
 (8, 6, 35890.9),
 (8, 7, 32892.6)]

In [153]:

# mesafeler listesini kullanmayı planlıyoruz (bizim durumumuzda süreler), bu yüzden `distances = dist list` parametresiyle başlatıyoruz.
fitness_dists = mlrose.TravellingSales(distances = dist_array)

In [154]:
problem_fit = mlrose.TSPOpt(length = length, fitness_fn = fitness_dists,
                            maximize=False)

In [155]:

mlrose.genetic_alg(problem_fit, random_state = 2)

(array([5, 0, 6, 3, 7, 4, 8, 1, 2]), 157848.0)

In [156]:

# Genetik algoritmayı kullanarak işlem yapıyoruz
best_state, best_fitness = mlrose.genetic_alg(problem_fit, random_state = 2)

In [157]:
print(f"The best state found is: {best_state}, taking {best_fitness} ({str(datetime.timedelta(seconds=best_fitness))})")

The best state found is: [5 0 6 3 7 4 8 1 2], taking 157848.0 (1 day, 19:50:48)


In [158]:

# iyi ama daha fazla yoğun çözümler için
best_state, best_fitness = mlrose.genetic_alg(problem_fit, mutation_prob = 0.2,  max_attempts = 500, random_state = 2)

In [159]:
print(f"The best state found is: {best_state}, taking {best_fitness} ({str(datetime.timedelta(seconds=best_fitness))})")

The best state found is: [6 3 7 4 8 1 2 5 0], taking 157848.0 (1 day, 19:50:48)


best_state, ziyaret edilecek yerlerin sırasını içerir. Görmek için şehirler listemize, onları ziyaret edebileceğimiz sıraya eşleyelim.

In [160]:
orders = {city: order for order, city in enumerate(best_state)}
orders

{6: 0, 3: 1, 7: 2, 4: 3, 8: 4, 1: 5, 2: 6, 5: 7, 0: 8}

In [162]:
ce_cities["order"] = ce_cities.index.map(orders)
ce_cities = ce_cities.sort_values(by="order")
ce_cities

,Country,capital,lat,lon,code,continent,order
6,Slovakia,Bratislava,48.150000,17.116667,SK,Europe,0
3,Hungary,Budapest,47.500000,19.083333,HU,Europe,1
7,Slovenia,Ljubljana,46.050000,14.516667,SI,Europe,2
4,Liechtenstein,Vaduz,47.133333,9.516667,LI,Europe,3
8,Switzerland,Bern,46.916667,7.466667,CH,Europe,4
1,Czech Republic,Prague,50.083333,14.466667,CZ,Europe,5
2,Germany,Berlin,52.516667,13.400000,DE,Europe,6
5,Poland,Warsaw,52.250000,21.000000,PL,Europe,7
0,Austria,Vienna,48.200000,16.366667,AT,Europe,8


Mesafenin gerçekten best_fitness olduğunu onaylayalım. Sıraya bağlı olarak next_city sütununu ekleyeceğiz ve özellikle son şehri  == 0 (veya minimum ) ile takip eden son şehri ele alacağız.

In [163]:
ce_cities["next_city"] = ce_cities["capital"].shift(-1)

# son bağlantı, son şehir ile ilk şehir arasında
ce_cities.loc[ce_cities["order"] == max(ce_cities["order"]), "next_city"] = ce_cities.loc[ce_cities["order"] == min(ce_cities["order"]), "capital"].values[0]
ce_cities

,Country,capital,lat,lon,code,continent,order,next_city
6,Slovakia,Bratislava,48.150000,17.116667,SK,Europe,0,Budapest
3,Hungary,Budapest,47.500000,19.083333,HU,Europe,1,Ljubljana
7,Slovenia,Ljubljana,46.050000,14.516667,SI,Europe,2,Vaduz
4,Liechtenstein,Vaduz,47.133333,9.516667,LI,Europe,3,Bern
8,Switzerland,Bern,46.916667,7.466667,CH,Europe,4,Prague
1,Czech Republic,Prague,50.083333,14.466667,CZ,Europe,5,Berlin
2,Germany,Berlin,52.516667,13.400000,DE,Europe,6,Warsaw
5,Poland,Warsaw,52.250000,21.000000,PL,Europe,7,Vienna
0,Austria,Vienna,48.200000,16.366667,AT,Europe,8,Bratislava


Sonra mesafeleri Distance_df'den saniyeler içinde birleştiriyoruz

In [164]:
ordered_ce_cities = ce_cities.merge(distances_df[["origin_name","destination_name","duration(s)"]], left_on=["capital","next_city"], right_on=["origin_name","destination_name"], how="left")

In [165]:

ordered_ce_cities

,Country,capital,lat,lon,code,continent,order,next_city,origin_name,destination_name,duration(s)
0,Slovakia,Bratislava,48.150000,17.116667,SK,Europe,0,Budapest,Bratislava,Budapest,7913.2
1,Hungary,Budapest,47.500000,19.083333,HU,Europe,1,Ljubljana,Budapest,Ljubljana,17602.4
2,Slovenia,Ljubljana,46.050000,14.516667,SI,Europe,2,Vaduz,Ljubljana,Vaduz,25030.4
3,Liechtenstein,Vaduz,47.133333,9.516667,LI,Europe,3,Bern,Vaduz,Bern,10100.8
4,Switzerland,Bern,46.916667,7.466667,CH,Europe,4,Prague,Bern,Prague,30125.8
5,Czech Republic,Prague,50.083333,14.466667,CZ,Europe,5,Berlin,Prague,Berlin,13625.0
6,Germany,Berlin,52.516667,13.400000,DE,Europe,6,Warsaw,Berlin,Warsaw,20523.7
7,Poland,Warsaw,52.250000,21.000000,PL,Europe,7,Vienna,Warsaw,Vienna,29661.1
8,Austria,Vienna,48.200000,16.366667,AT,Europe,8,Bratislava,Vienna,Bratislava,3484.5


Bu mesafenin toplamının gerçekten, Gezgin Satıcı Probleminin best_fitness olarak tanımladığı şey olup olmadığını kontrol edelim.

In [166]:
ordered_ce_cities["duration(s)"].sum()

158066.90000000002

Rota döngüsel olduğu için başkentlerin herhangi birinden başlayabiliriz. Minimum seyahat süresini elde etmek için, en uzun sürenin takip ettiği şehirde bitirilebilir.

In [167]:
ordered_ce_cities["duration(s)"].max()

30125.8

In [168]:
ordered_ce_cities["duration(s)"].min()

3484.5

In [169]:
ordered_ce_cities.style.highlight_max(color = 'red', axis = 0, subset="duration(s)")

,Country,capital,lat,lon,code,continent,order,next_city,origin_name,destination_name,duration(s)
0,Slovakia,Bratislava,48.150000,17.116667,SK,Europe,0,Budapest,Bratislava,Budapest,7913.200000
1,Hungary,Budapest,47.500000,19.083333,HU,Europe,1,Ljubljana,Budapest,Ljubljana,17602.400000
2,Slovenia,Ljubljana,46.050000,14.516667,SI,Europe,2,Vaduz,Ljubljana,Vaduz,25030.400000
3,Liechtenstein,Vaduz,47.133333,9.516667,LI,Europe,3,Bern,Vaduz,Bern,10100.800000
4,Switzerland,Bern,46.916667,7.466667,CH,Europe,4,Prague,Bern,Prague,30125.800000
5,Czech Republic,Prague,50.083333,14.466667,CZ,Europe,5,Berlin,Prague,Berlin,13625.000000
6,Germany,Berlin,52.516667,13.400000,DE,Europe,6,Warsaw,Berlin,Warsaw,20523.700000
7,Poland,Warsaw,52.250000,21.000000,PL,Europe,7,Vienna,Warsaw,Vienna,29661.100000
8,Austria,Vienna,48.200000,16.366667,AT,Europe,8,Bratislava,Vienna,Bratislava,3484.500000


In [170]:
ordered_ce_cities.style.highlight_min(color = 'blue', axis = 0, subset="duration(s)")

,Country,capital,lat,lon,code,continent,order,next_city,origin_name,destination_name,duration(s)
0,Slovakia,Bratislava,48.150000,17.116667,SK,Europe,0,Budapest,Bratislava,Budapest,7913.200000
1,Hungary,Budapest,47.500000,19.083333,HU,Europe,1,Ljubljana,Budapest,Ljubljana,17602.400000
2,Slovenia,Ljubljana,46.050000,14.516667,SI,Europe,2,Vaduz,Ljubljana,Vaduz,25030.400000
3,Liechtenstein,Vaduz,47.133333,9.516667,LI,Europe,3,Bern,Vaduz,Bern,10100.800000
4,Switzerland,Bern,46.916667,7.466667,CH,Europe,4,Prague,Bern,Prague,30125.800000
5,Czech Republic,Prague,50.083333,14.466667,CZ,Europe,5,Berlin,Prague,Berlin,13625.000000
6,Germany,Berlin,52.516667,13.400000,DE,Europe,6,Warsaw,Berlin,Warsaw,20523.700000
7,Poland,Warsaw,52.250000,21.000000,PL,Europe,7,Vienna,Warsaw,Vienna,29661.100000
8,Austria,Vienna,48.200000,16.366667,AT,Europe,8,Bratislava,Vienna,Bratislava,3484.500000


Gördüğünüz gibi, bizim durumumuzda, Berlin'de başlayıp Prauge'da bitirebilirseniz, mesafe en küçük olacaktır.

In [171]:
duration_s = ordered_ce_cities["duration(s)"].sum() - ordered_ce_cities["duration(s)"].max()
duration_s, str(datetime.timedelta(seconds=duration_s))

(127941.10000000002, '1 day, 11:32:21.100000')

## Bunun gerçekten kısaltılmış olası yol seçimi olduğunu onaylayalım

In [172]:
l = [0,1,2,3,4,5,6,7,8]

In [173]:
# There's possible (n)!/2 combinations; 1/2 because path 1-2-3 is the same as 3-2-1
math.factorial(length+2)/2

19958400.0

Tüm olası yolları oluşturduk

In [174]:
perm = [l for l in itertools.permutations(range(9), 9)]
len(perm)

362880

In [175]:
perm[:5]

[(0, 1, 2, 3, 4, 5, 6, 7, 8),
 (0, 1, 2, 3, 4, 5, 6, 8, 7),
 (0, 1, 2, 3, 4, 5, 7, 6, 8),
 (0, 1, 2, 3, 4, 5, 7, 8, 6),
 (0, 1, 2, 3, 4, 5, 8, 6, 7)]

In [176]:
distances = {(int(d[0]), int(d[1])):d[2] for d in distances_df[["origin","destination","duration(s)"]].values.tolist()}
distances[(0,1)]

13485.7

In [177]:
# pick the first path
path = perm[0]

# add th first element to conclude the circular path
path = path + (path[0],)

# iterate through the path and sum all the distances
total_path_distance = 0
for i in range(len(path)-1):
    edge = (path[i], path[i+1])
    total_path_distance += distances[edge]
    print(edge, distances[edge])
print(total_path_distance)

# list comprehension
sum([distances[(path[i],path[i+1])] for i in range(len(path)-1)])

(0, 1) 13485.7
(1, 2) 13625.0
(2, 3) 33090.8
(3, 4) 33404.8
(4, 5) 45070.2
(5, 6) 28368.8
(6, 7) 16745.4
(7, 8) 32862.3
(8, 0) 32903.3
249556.3


249556.3

In [178]:
mn = np.inf
min_paths = []

for p in perm:
    
    # add th first element to conclude the circular path
    p = p + (p[0],)
    
    total_path_distance = sum([distances[(p[i],p[i+1])] for i in range(len(path)-1)])
    
    if total_path_distance < mn:
        mn = total_path_distance
        min_paths = [p]
    elif total_path_distance == mn:
        min_paths.append(p)
    else:
        pass
    
print(mn, len(min_paths), min_paths)

157645.09999999998 1 [(5, 2, 1, 8, 4, 7, 3, 6, 0, 5)]


In [179]:
def path_to_df(path, cities, distances_df):
    orders = {city: order for order, city in enumerate(path[0])}
    
    cities["order"] = cities.index.map(orders)
    cities = cities.sort_values(by="order")
    
    cities["next_city"] = cities["capital"].shift(-1)
    cities["prev_city"] = cities["capital"].shift(1)

    # the last connection is between the last city and the first one
    cities.loc[cities["order"] == max(cities["order"]), "next_city"] = cities.loc[cities["order"] == min(cities["order"]), "capital"].values[0]
    cities.loc[cities["order"] == min(cities["order"]), "prev_city"] = cities.loc[cities["order"] == max(cities["order"]), "capital"].values[0]
    
    ordered_ce_cities = cities.merge(distances_df[["origin_name","destination_name","duration(s)"]], left_on=["capital","next_city"], right_on=["origin_name","destination_name"], how="left")
    
    return ordered_ce_cities["duration(s)"].sum(), ordered_ce_cities

In [180]:
total_duration, path_df = path_to_df(min_paths, ce_cities, distances_df)
path_df.style.highlight_max(color = 'red', axis = 0, subset="duration(s)")

,Country,capital,lat,lon,code,continent,order,next_city,prev_city,origin_name,destination_name,duration(s)
0,Germany,Berlin,52.516667,13.400000,DE,Europe,1,Prague,Warsaw,Berlin,Prague,13652.600000
1,Czech Republic,Prague,50.083333,14.466667,CZ,Europe,2,Bern,Berlin,Prague,Bern,30028.300000
2,Switzerland,Bern,46.916667,7.466667,CH,Europe,3,Vaduz,Prague,Bern,Vaduz,10012.400000
3,Liechtenstein,Vaduz,47.133333,9.516667,LI,Europe,4,Ljubljana,Bern,Vaduz,Ljubljana,24886.900000
4,Slovenia,Ljubljana,46.050000,14.516667,SI,Europe,5,Budapest,Vaduz,Ljubljana,Budapest,17640.200000
5,Hungary,Budapest,47.500000,19.083333,HU,Europe,6,Bratislava,Ljubljana,Budapest,Bratislava,7880.200000
6,Slovakia,Bratislava,48.150000,17.116667,SK,Europe,7,Vienna,Budapest,Bratislava,Vienna,3488.700000
7,Austria,Vienna,48.200000,16.366667,AT,Europe,8,Warsaw,Bratislava,Vienna,Warsaw,29590.600000
8,Poland,Warsaw,52.250000,21.000000,PL,Europe,9,Berlin,Vienna,Warsaw,Berlin,20465.200000


In [181]:
total_duration

157645.10000000003

Yolun Berlin'den başlaması için sırayı değiştiriyoruz.

In [182]:
path_df_reversed = path_df.sort_values(by="order").reset_index(drop=True)
path_df_reversed = path_df_reversed.reindex(index=path_df_reversed.index[::-1])
path_df_reversed.rename(columns={"destination_name":"origin_name", "origin_name":"destination_name"}, inplace=True)
path_df_reversed[["order", "origin_name", "destination_name", "duration(s)"]]

,order,origin_name,destination_name,duration(s)
8,9,Berlin,Warsaw,20465.2
7,8,Warsaw,Vienna,29590.6
6,7,Vienna,Bratislava,3488.7
5,6,Bratislava,Budapest,7880.2
4,5,Budapest,Ljubljana,17640.2
3,4,Ljubljana,Vaduz,24886.9
2,3,Vaduz,Bern,10012.4
1,2,Bern,Prague,30028.3
0,1,Prague,Berlin,13652.6


Bu yol, Gezgin Satıcı algoritmamızın bulduğuyla tamamen aynıdır. min_paths değişkeninde, bu yolların tüm kombinasyonlarının en kısa olduğunu görebilirsiniz (başladığımız yere geri döndüğümüz düşünülürse).

Plotly Geochart ile görsel

In [74]:
import plotly.graph_objects as go

In [185]:
# başkentlerin çizilmesi
fig = go.Figure(data=go.Scattergeo(
    locationmode = 'USA-states',
        lon = path_df['lon'],
        lat = path_df['lat'],
        text = df['capital'],
        mode = 'markers',
    name="capitals"
        ))

# draw the paths between the capitals
for i in range(len(path_df)-1):
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon=[path_df.loc[i,"lon"],path_df.loc[i+1,"lon"]], 
        lat=[path_df.loc[i,"lat"],path_df.loc[i+1,"lat"]], 
        name="-".join([path_df.loc[i,"capital"],path_df.loc[i+1,"capital"]]),
        mode="lines"))
    
# the last path
fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon=[path_df.loc[8,"lon"],path_df.loc[0,"lon"]], 
        lat=[path_df.loc[8,"lat"],path_df.loc[0,"lat"]], 
        name="-".join([path_df.loc[8,"capital"],path_df.loc[0,"capital"]]),
        mode="lines"))

fig.update_layout(
        title = 'Shortest Route Between Central European Cities',
        geo_scope='europe',
    )
fig.show()
